In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
cd /content/drive/MyDrive/mygithub/pytorchForAudioPractice

/content/drive/.shortcut-targets-by-id/1j0tKfNJ74iEAtyLmY4PY6L2fktF9jL1I/mygithub/pytorchForAudioPractice


In [15]:
!pip install torch torchaudio torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.autograd.profiler as profiler

In [17]:
def download_mnist_datasets():
  train_data = datasets.MNIST(train=True, download=True, transform=ToTensor(), root="data")
  validate_data = datasets.MNIST(train=False, download=True, transform=ToTensor(), root="data")
  return train_data, validate_data


In [18]:
from torch.nn.modules.linear import Linear
class FeedForwardNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.denseLayers = nn.Sequential(
        nn.Linear(28*28,256), # image size 28x28
        nn.ReLU(),
        nn.Linear(256,10) # 10 classes 0 to 9 
    )
    self.activation = nn.Softmax(dim=1)

  def forward(self, inputData):

    flattenedData = self.flatten(inputData)
    logits = self.denseLayers(flattenedData)
    predictions = self.activation(logits)

    return predictions

In [19]:
def trainOneEpoch(model, dataLoader, lossFn, optimiser, device):


  for input, target in dataLoader:

    input, target = input.to(device), target.to(device)

    #calculate loss
    predication = model(input)

    loss = lossFn(predication, target)

    # backpropagation and weight update

    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

  print(f"loss: {loss.item()}")

In [20]:
def train(model, dataLoader, lossFn, optimiser, device, epochs):
  for i in range(epochs):

    print(f"Epoch {i+1}")
    trainOneEpoch(model, dataLoader, lossFn, optimiser, device)
    print("---------------------------")
  
  print("Finished training")

# **Model Train**

In [21]:
BATCH_SIZE = 128
LEARNING_RATE = 0.001
EPOCHS = 10

# Download data
train_data, _ = download_mnist_datasets()

# data load
dataLoader = DataLoader(train_data, batch_size=BATCH_SIZE)

# Build Model

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(f"Using {device}")

model = FeedForwardNN().to(device)

# Train

lossFn = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)


train(model, dataLoader, lossFn, optimiser, device, EPOCHS)

# Save Model

torch.save(model.state_dict(),"modelNN.pth")
print("Trained feed forward net saved at modelNN.pth")



Using cpu
Epoch 1
loss: 1.5117287635803223
---------------------------
Epoch 2
loss: 1.49992835521698
---------------------------
Epoch 3
loss: 1.4886845350265503
---------------------------
Epoch 4
loss: 1.4811148643493652
---------------------------
Epoch 5
loss: 1.475595474243164
---------------------------
Epoch 6
loss: 1.473410964012146
---------------------------
Epoch 7
loss: 1.4738837480545044
---------------------------
Epoch 8
loss: 1.4731812477111816
---------------------------
Epoch 9
loss: 1.4724453687667847
---------------------------
Epoch 10
loss: 1.4728208780288696
---------------------------
Finished training
Trained feed forward net saved at modelNN.pth


# Model Infere

In [26]:
CLASS_MAPPING = [
    "0",
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9"
]

In [29]:
def predict(model, input, target, classMapping):
  # switch eval 
  model.eval()

  # run infere without gradient evaluation
  with torch.no_grad():
    predictions = model(input)

    # Tensor (1, 10) --> [[0.1, 0.001, .... , 0.8]]
    predictedIndex = predictions[0].argmax(0)

    predicted = classMapping[predictedIndex]
    expected = classMapping[target]

  return  predicted, expected

In [30]:
# Load Validate Data
_, validatingData = download_mnist_datasets()

# Load Model

feedForwardNetObj = FeedForwardNN()

dictStatLoaded = torch.load("modelNN.pth")

feedForwardNetObj.load_state_dict(dictStatLoaded)

# get sample from the validating data
input, target = validatingData[0][0], validatingData[0][1]

# Make inference

predicted, expected = predict(feedForwardNetObj, input, target, CLASS_MAPPING)

print(f"Predicted: '{predicted}', expected: '{expected}'")

Predicted: '7', expected: '7'


# Customized Urban Dataset

https://urbansounddataset.weebly.com/urbansound8k.html

# To download dataset

```
cd ---> path you need
!pip install opendatasets --upgrade --quiet

import opendatasets as od

dataset_url = 'https://goo.gl/8hY5ER'
od.download(dataset_url)
```

ref: https://jovian.ai/charmzshab/urban-sound-dataset

and also: https://www.kaggle.com/datasets/chrisfilo/urbansound8k

In [7]:
import os
from torch.utils.data import Dataset
import pandas as pd
import torchaudio

class UrbanSoundDataset(Dataset):

  def __init__(self, annotationsFile, audioDir):
    self.annotations = pd.read_csv(annotationsFile)
    self.audioDir = audioDir

  def __len__(self):
    return len(self.annotations)
    
  def __getitem__(self,index):
    audioSamplePath = self._getAudioSamplePath(index)
    label = self._getAudioSampleLabel(index)
    signal, sr = torchaudio.load(audioSamplePath)
    return signal, label

  def _getAudioSamplePath(self, index):
    fold = f"fold{self.annotations.iloc[index,5]}" # 5 where fold is located he cloumb # 5
    path = os.path.join(self.audioDir, fold, self.annotations.iloc[index, 0]) # where 0 is the raw of the .wav files names
    return path

  def _getAudioSampleLabel(self, index):
    return self.annotations.iloc[index, 6] # where the raw #6 is the classId

In [8]:
ANNOTATION_FILE = "/content/drive/MyDrive/mygithub/pytorchForAudioPractice/data/UrbanSound8K/metadata/UrbanSound8K.csv"
AUDIO_DIR = "/content/drive/MyDrive/mygithub/pytorchForAudioPractice/data/UrbanSound8K/audio"

usd = UrbanSoundDataset(ANNOTATION_FILE, AUDIO_DIR)

print(f"There are {len(usd)} samples in the dataset.")
signal, label = usd[0]



There are 8732 samples in the dataset.


In [12]:
label

3